# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1533316845.log
INFO:graphlab.cython.cy_server:GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1533316845.log


This non-commercial license of GraphLab Create for academic use is assigned to sunilpatil423@gmail.com and will expire on July 02, 2019.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.751495     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.799285     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [9]:
model_all.coefficients.print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [10]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [12]:
import numpy as np

In [17]:
RSS_array = []
for l1_penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                          validation_set=None, 
                                          l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    error = (validation['price'] - model.predict(validation))
    residual_squared = error**2
    RSS = residual_squared.sum()
    RSS_array.append(RSS)
    
    print "l1_penalty:%d\tRSS:%d" %(l1_penalty,RSS)

l1_penalty:10	RSS:625766285142461
l1_penalty:31	RSS:625766285362395
l1_penalty:100	RSS:625766286057887
l1_penalty:316	RSS:625766288257224
l1_penalty:1000	RSS:625766295212186
l1_penalty:3162	RSS:625766317206077
l1_penalty:10000	RSS:625766386760661
l1_penalty:31622	RSS:625766606749281
l1_penalty:100000	RSS:625767302791633
l1_penalty:316227	RSS:625769507643884
l1_penalty:1000000	RSS:625776517727025
l1_penalty:3162277	RSS:625799062845466
l1_penalty:10000000	RSS:625883719085424


In [78]:
print min(RSS_array)
print len(RSS_array)

1.04419689085e+15
16


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [79]:
l1_penalty = 10
model_best_l1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                          validation_set=None, 
                                          l2_penalty=0., l1_penalty=l1_penalty, verbose=False)

test_error = testing['price'] - model_best_l1.predict(testing)
error_squared = test_error**2
RSS = error_squared.sum()

print RSS

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [80]:
model_best_l1.coefficients.print_rows(num_rows=20)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [23]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [24]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [41]:
l1_penalty_nnz = []
first_min_loop = 0
first_max_loop = 0

for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                          validation_set=None, 
                                          l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    
    non_zero_weights = (model['coefficients']['value']).nnz()
    l1_penalty_nnz.append(non_zero_weights)
    
    if (non_zero_weights > max_nonzeros):
        if (first_min_loop == 0):
            l1_penalty_min = l1_penalty
            first_min_loop = 1        
        elif (l1_penalty_min < l1_penalty):
            l1_penalty_min = l1_penalty
            
    elif (non_zero_weights <= max_nonzeros):
        if (first_max_loop == 0):
            l1_penalty_max = l1_penalty
            first_max_loop = 1         
        elif (l1_penalty_max > l1_penalty):
            l1_penalty_max = l1_penalty

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [42]:
print l1_penalty_values
print l1_penalty_nnz
print l1_penalty_min
print l1_penalty_max

[  2.97635144e+09   3.01931664e+09   3.06228184e+09   3.10524703e+09
   3.14821223e+09   3.19117743e+09   3.23414263e+09   3.27710782e+09
   3.32007302e+09   3.36303822e+09   3.40600341e+09   3.44896861e+09
   3.49193381e+09   3.53489901e+09   3.57786420e+09   3.62082940e+09
   3.66379460e+09   3.70675980e+09   3.74972499e+09   3.79269019e+09]
[10, 10, 10, 10, 10, 10, 10, 10, 8, 8, 8, 7, 7, 7, 7, 6, 6, 6, 6, 6]
3406003414.84
3448968612.16


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [62]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [69]:
RSS_array = []
l1_penalty_list = []

for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                          validation_set=None, 
                                          l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    
    non_zero_weights = (model['coefficients']['value']).nnz()

    if (non_zero_weights == max_nonzeros):
        error = (validation['price'] - model.predict(validation))
        residual_squared = error**2
        RSS = residual_squared.sum()
        RSS_array.append(RSS)
        l1_penalty_list.append(l1_penalty)

        print "l1_penalty:%d\tRSS:%d\tnon_zero_weights:%d" %(l1_penalty,RSS, non_zero_weights)

l1_penalty:3415048719	RSS:1044196890847944	non_zero_weights:7
l1_penalty:3417310045	RSS:1044378071619645	non_zero_weights:7
l1_penalty:3419571371	RSS:1044559318256503	non_zero_weights:7
l1_penalty:3421832698	RSS:1044740589724656	non_zero_weights:7
l1_penalty:3424094024	RSS:1044921906538465	non_zero_weights:7
l1_penalty:3426355350	RSS:1045103289232852	non_zero_weights:7
l1_penalty:3428616676	RSS:1045284696743117	non_zero_weights:7
l1_penalty:3430878002	RSS:1045466149599047	non_zero_weights:7
l1_penalty:3433139328	RSS:1045647668350943	non_zero_weights:7
l1_penalty:3435400655	RSS:1045829211903325	non_zero_weights:7
l1_penalty:3437661981	RSS:1046010800801369	non_zero_weights:7
l1_penalty:3439923307	RSS:1046192435045071	non_zero_weights:7
l1_penalty:3442184633	RSS:1046374135205278	non_zero_weights:7
l1_penalty:3444445959	RSS:1046555860145431	non_zero_weights:7
l1_penalty:3446707285	RSS:1046742454631130	non_zero_weights:7
l1_penalty:3448968612	RSS:1046937488751715	non_zero_weights:7


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [70]:
print min(RSS_array)

1.04419689085e+15


In [71]:
print RSS_array.index(min(RSS_array))

0


In [74]:
print l1_penalty_list[RSS_array.index(min(RSS_array))]
l1_penalty = l1_penalty_list[RSS_array.index(min(RSS_array))]

3415048719.54


In [75]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                      validation_set=None, 
                                      l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
model.coefficients.print_rows(num_rows=18)    

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 220419.096835 |  None  |
|     bedrooms     |  None | 782.988241194 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 16044.2089882 |  None  |
|   sqft_living    |  None | 32.5565458322 |  None  |
| sqft_living_sqrt |  None |  698.27080718 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2954.13888897 |  None  |
|    sqft_above    |  None | 30.2313633728 |  None  |
|  sqft_basement   |  None |